In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import string


In [ ]:
data=pd.read_csv("D:/softwares/deployement/netflix-tiles-master/netflix_titles.csv")

In [ ]:
data.head()

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

In [ ]:
data.describe()


In [ ]:
data.describe(include="all")

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(data.isnull())

In [ ]:
data.country.mode()

In [ ]:
data["country"]=data["country"].fillna("United States")

In [ ]:
data["director"]=data.director.fillna("")


In [ ]:
new_data=data.dropna()

In [ ]:
data.shape

In [ ]:
new_data.shape

In [ ]:
new_data.isnull().sum()

In [ ]:
new_data.head()

In [ ]:
new_data.director

In [ ]:
new_data.rating.value_counts()

In [ ]:
new_data["rating"]=new_data["rating"].map({"TV-MA":"Adults","TV-14":"Teens","TV-PG":"Older Kids","R":"Adults","PG-13":"Teens","TV-Y7":"Older Kids","TV-Y":"Kids","PG":"Older Kids","TV-G":"Kids","NR":"Adults","G":"Kids","UR":"Adults","TV-Y7-FV":"Older Kids","NC-17":"Adults"})

In [ ]:
new_data.head()

In [ ]:
sns.set_style("whitegrid")
sns.countplot("type",data=data)

In [ ]:
month=[]
for row in new_data["date_added"]:
    month.append(row.split()[0])

new_data["data_added_month"]=month

In [ ]:
date=[]
for row in new_data["date_added"]:
    date.append(row.split()[1].split(",")[0])
    
new_data["date_added_date"]=date

In [ ]:
year=[]
for row in new_data["date_added"]:
    year.append(row.split()[2])
    
new_data["date_added_year"]=year

In [ ]:
new_data.head()

In [ ]:
new_data.columns

In [ ]:
new_data.drop(["show_id","date_added"],axis=1,inplace=True)

In [ ]:
new_data.head()

In [ ]:
new_data.dtypes

In [ ]:
new_data["release_year"]=new_data.release_year.astype("string")

In [ ]:
new_data.dtypes

In [ ]:
new_data.head()

In [ ]:
feature=new_data[["type","title","director","cast","country","rating","listed_in","description"]]

In [ ]:
feature.head()

In [ ]:
sns.heatmap(feature.isnull())

In [ ]:
feature_data=feature["type"]+" "+feature["director"]+" "+feature["cast"]+" "+feature["country"]+" "+feature["rating"]+" "+feature["listed_in"]+" "+feature["description"]

In [ ]:
feature["feature_data"]=feature_data

In [ ]:
feature.head()

In [ ]:
df=feature["feature_data"]

In [ ]:
df.head()

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
word=[]
for row in df:
    words=word_tokenize(row)
    word.append(words)

In [ ]:
feature["word_tokenize"]=word

In [ ]:
feature.head()

In [ ]:
from nltk.corpus import stopwords


In [ ]:
print(stopwords.words("english"))

In [ ]:
print(string.punctuation)

In [ ]:
def text_processing(row):
    non_punc= [word.lower() for word in row if word not in string.punctuation]
    non_punc=[word for word in non_punc if word not in stopwords.words("english")]
    return " ".join(non_punc)


In [ ]:
feature["clean_data"]=feature["word_tokenize"].apply(text_processing)


In [ ]:
feature.head()

In [ ]:
feature.insert(0,"index",feature.index)

In [ ]:
feature.head()

In [ ]:
feature.clean_data[0]

In [ ]:
model=CountVectorizer()

In [ ]:
count_matrix=model.fit_transform(feature["clean_data"])

In [ ]:
cosine_sim=cosine_similarity(count_matrix)

In [ ]:
def get_title(index):
    return feature[feature.index==index]["title"].values[0]
def get_index(title):
    return feature[feature.title==title]["index"].values[0]

In [ ]:
import pickle

In [ ]:
pickle.dump(model,open("netflix_count_vectorizer.pkl","wb"))

In [ ]:
show_user_likes="21"
show_index=get_index(show_user_likes)
similar_shows=list(enumerate(cosine_sim[show_index]))

In [ ]:
similar_shows

In [ ]:
sorted_similar_shows=sorted(similar_shows,key=lambda x:x[1],reverse=True)[1:]

In [ ]:
sorted_similar_shows

In [ ]:
i=0
print(f"Top 5 similar shows {show_user_likes} are :")
for element in sorted_similar_shows:
    print(get_title(element[0]))
    i=i+1
    if i>5:
        break

In [ ]:
feature[feature["title"]=="Hairspray"]


In [ ]:
feature[feature["title"]=="Sugar High"]


In [ ]:
feature[feature["title"]=="Giving Voice"]


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 

In [ ]:
model1=TfidfVectorizer(use_idf = True,lowercase = True, strip_accents='ascii',stop_words=set(stopwords.words("english")))

In [ ]:
vectorizer=model1.fit_transform(feature["clean_data"])

In [ ]:
cosine_similarity=cosine_similarity(vectorizer)

In [ ]:
show_user_likes="21"
show_index=get_index(show_user_likes)
similar_shows=list(enumerate(cosine_sim[show_index]))

In [ ]:
similar_shows

In [ ]:
sorted_similar_shows=sorted(similar_shows,key=lambda x:x[1],reverse=True)[1:]

In [ ]:
sorted_similar_shows

In [ ]:
i=0
print(f"Top 5 similar shows {show_user_likes} are :")
for element in sorted_similar_shows:
    print(get_title(element[0]))
    i=i+1
    if i>5:
        break

In [ ]:
feature.head()

In [ ]:
feature.columns

In [ ]:
new=feature[["index","type","title","director","cast","rating","listed_in","description","clean_data"]]

In [ ]:
new.head()

In [ ]:
new.to_csv("main.csv",index=False)

In [ ]:
s=pd.read_csv("main.csv")

In [ ]:
s.head()